<a href="https://colab.research.google.com/github/hyunaeee/hongong/blob/main/face_ai_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -U autotrain-advanced

In [2]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [ ]:
import os

model_name = 'stabilityai/stable-diffusion-xl-base-1.0'
prompt = "photo of a {keyword}" #Need_to_fix_a_keyword
output_path = "/content/drive/MyDrive"
image_path = "/content/drive/MyDrive/{keyword}" #Need_to_fix_a_keyword

learning_rate = 1e-4
num_steps = 1000
batch_size = 2
gradient_accumulation = 4
resolution = 1024
use_8bit_adam = True
use_xformers = True
use_fp16 = True
train_text_encoder = False
gradient_checkpointing = True

os.environ["MODEL_NAME"] = model_name
os.environ["PROJECT_NAME"] = output_path
os.environ["PROMPT"] = prompt
os.environ["IMAGE_PATH"] = image_path
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["USE_8BIT_ADAM"] = str(use_8bit_adam)
os.environ["USE_XFORMERS"] = str(use_xformers)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["GRADIENT_CHECKPOINTING"] = str(gradient_checkpointing)

In [ ]:
!autotrain dreambooth \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--image-path ${IMAGE_PATH} \
--prompt "${PROMPT}" \
--resolution ${RESOLUTION} \
--batch-size ${BATCH_SIZE} \
--num-steps ${NUM_STEPS} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--lr ${LEARNING_RATE} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_XFORMERS" == "True" ]] && echo "--xformers" ) \
$( [[ "$USE_8BIT_ADAM" == "True" ]] && echo "--use-8bit-adam" ) \
$( [[ "$GRADIENT_CHECKPOINTING" == "True" ]] && echo "--gradient-checkpointing" )

In [ ]:
!pip install invisible_watermark transformers accelerate safetensors
!pip install git+https://github.com/huggingface/diffusers
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch
model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
model,
torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.load_lora_weights("/content/drive/MyDrive/{filename}.safetensors")
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
"stabilityai/stable-diffusion-xl-refiner-1.0",
torch_dtype=torch.float16,
)
refiner.to("cuda")

In [ ]:
prompt = "A {keyword} man, photorealistic, bold and elegant, close up, sharp expression, look at camera, block noise, vibrant colors, glitchcore, fuzz, color bars, abstract background, pop kei, bold outlines, ultrafine detailed, best quality"
negative_prompt = "ugly, deformed, noisy, blurry, distorted, grainy, text, cropped"
#for seed in range(10):
generator = torch.Generator("cuda").manual_seed(43)
image = pipe(prompt=prompt, negative_prompt=negative_prompt, generator=generator, num_inference_steps=40)
image = image.images[0]
image = refiner(prompt=prompt, negative_prompt=negative_prompt, generator=generator, image=image)
image = image.images[0]